In [68]:
from bs4 import BeautifulSoup
import os
from IPython.display import display

def create_dataframe(relative_path, csv_sub_folder):
    folder_path = r"..\..\his_usage_data\nba_usage_historic\nba_html_2019-20"  #f"schedule/nba_schedules/nba_html_{year}"            #nba_html_2019-20 

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        #print(file_path)

        # Check if it's a file (and not a subfolder)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                # Read the content of the file
                html_content = file.read()
                # removing the .html from the file name so I can .csv when it comes time to save
                original_filename = filename
                remove_string_in_filename = ".html"
                modified_filename = original_filename.replace(remove_string_in_filename, "")
                print(f"Contents of {modified_filename}")

            # Parse the HTML content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Find the table header row
            header_row = soup.find('tr', class_='Crom_headers__mzI_m')

            # Extract the text from each <th> element
            if header_row:
                headers = [th.text.strip() for th in header_row.find_all('th')]
                del headers[26:]
                print(headers)
                print(len(headers))
            else:
                print(f"Header {modified_filename} row not found.")

            # rows = soup.find_all('tr')


            rows = soup.find_all('tbody', class_="Crom_body__UYOcU")

            extracted_data = []
            for row in rows:
                spans = row.find_all('td')
                row_text = [span.get_text().strip() for span in spans]
                for i in range (0, len(row_text), 26):
                    sublist = row_text[i:i+26]
                    extracted_data.append(sublist)

            print(extracted_data)



            import pandas as pd
            df = pd.DataFrame(extracted_data, columns=headers)
            pd.set_option('display.max_rows', 1000)  # Maximum number of rows to display
            pd.set_option('display.max_columns', None)  # Show all columns
            pd.set_option('display.width', 1000)  # Adjust column width for better readability

            df = df.rename(columns={'': 'RANK'})
            int_columns = ['RANK','AGE','GP','W','L']
            float_columns = [
                'MIN',  'USG%', '%FGM', '%FGA', '%3PM', 
                '%3PA', '%FTM', '%FTA', '%OREB', '%DREB', '%REB', 
                '%AST', '%TOV', '%STL', '%BLK', '%BLKA', '%PF', '%PFD', '%PTS']

            df[int_columns] = df[int_columns].astype(int)
            df[float_columns] = df[float_columns].astype(float)

            df.info()


            #save to csv
            path = f'nba_scheduled_csv/{csv_sub_folder}'
            csv_path = path
            os.makedirs(csv_path, exist_ok=True)
            df.to_csv(f"{csv_path}/{modified_filename}.csv", index=False)

            print(len(df))



Contents of 2019-20_content
['', 'Player', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'USG%', '%FGM', '%FGA', '%3PM', '%3PA', '%FTM', '%FTA', '%OREB', '%DREB', '%REB', '%AST', '%TOV', '%STL', '%BLK', '%BLKA', '%PF', '%PFD', '%PTS']
26
[['1', 'Malik Newman', 'CLE', '23', '1', '0', '1', '4.0', '57.1', '0.0', '50.0', '0.0', '50.0', '100', '100', '0.0', '0.0', '0.0', '0.0', '50.0', '0.0', '0.0', '0.0', '0.0', '100', '50.0'], ['2', 'Giannis Antetokounmpo', 'MIL', '25', '63', '51', '12', '1917.0', '36.3', '38.8', '34.0', '16.2', '18.7', '51.3', '57.7', '36.9', '40.9', '40.2', '33.9', '39.9', '20.7', '25.0', '40.1', '25.1', '54.1', '38.3'], ['3', 'James Harden', 'HOU', '30', '68', '43', '25', '2483.0', '35.6', '32.5', '33.0', '36.8', '36.1', '62.3', '57.7', '13.2', '20.9', '19.1', '46.4', '41.2', '27.8', '21.6', '34.8', '20.3', '47.5', '38.5'], ['4', 'Luka Dončić', 'DAL', '21', '61', '36', '25', '2047.0', '35.5', '32.9', '33.4', '26.4', '30.0', '48.7', '48.9', '17.4', '32.0', '28.7', '52.8', '46.2